# Import libraries

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


# Create SparkSession
spark = SparkSession.builder.appName('Song_Review_Analysis') \
        .master("local[1]").getOrCreate()

# Import Dataset

In [2]:
data = spark.read.csv("songsDataset.csv",header=True)
data

DataFrame['userID': string, 'songID': string, 'rating': string]

In [3]:
data = data.select(col("'userID'").alias("userID"), col("'songID'").alias("songID"), col("'rating'").alias("rating"))

# High Level Overview of Dataset

In [4]:
review_count_df = data.groupBy("userID").count()

In [5]:
review_count_df.show()

+------+-----+
|userID|count|
+------+-----+
|   296|   10|
|   467|   10|
|   675|   10|
|   691|   10|
|   829|   10|
|  1090|   10|
|  1159|   10|
|  1436|   10|
|  1512|   10|
|  1572|   10|
|  2069|   10|
|  2088|   10|
|  2136|   10|
|  2162|   10|
|  2294|   10|
|  2904|   10|
|  3210|   10|
|  3414|   10|
|  3606|   10|
|  3959|   10|
+------+-----+
only showing top 20 rows



In [6]:
review_count_df.filter(review_count_df.userID == 199999).show()

+------+-----+
|userID|count|
+------+-----+
|199999|   10|
+------+-----+



# Positive Interaction Filtering

In [7]:
positive_rating_data = data.filter(data.rating >= 3)

positive_rating_data.count()

1412937

In [8]:
positive_review_count_df = positive_rating_data.groupBy("userID").count()

positive_review_count_df = positive_review_count_df.select(col("userID"), col("count").alias("review_count"))

positive_review_count_df.show()

+------+------------+
|userID|review_count|
+------+------------+
|   296|           6|
|   467|           5|
|   675|           6|
|   691|          10|
|   829|           9|
|  1159|           3|
|  1436|           7|
|  1512|           5|
|  1572|           4|
|  2069|           7|
|  2088|           7|
|  2136|           5|
|  2162|           9|
|  2294|           9|
|  2904|           7|
|  3210|           7|
|  3414|          10|
|  3606|          10|
|  3959|           8|
|  4032|           9|
+------+------------+
only showing top 20 rows



In [9]:
positive_review_count_df.sort("review_count").show()

+------+------------+
|userID|review_count|
+------+------------+
|140876|           1|
| 59680|           1|
|141168|           1|
| 42370|           1|
|144656|           1|
|121867|           1|
|148664|           1|
| 67196|           1|
|151772|           1|
| 23318|           1|
|163886|           1|
| 66584|           1|
|167438|           1|
|109514|           1|
|173637|           1|
|  1394|           1|
| 68471|           1|
|  3368|           1|
| 43462|           1|
| 15052|           1|
+------+------------+
only showing top 20 rows



# Filter only those users who rated positively at least 5 songs

In [12]:
active_positive_review_count_df = positive_review_count_df.filter(positive_review_count_df.review_count >= 5)

In [15]:
final_data = active_positive_review_count_df.join(positive_rating_data, how="left", on="userID")

In [16]:
final_data.count()

1315321

In [17]:
final_data.show()

+------+------------+------+------+
|userID|review_count|songID|rating|
+------+------------+------+------+
|  1436|           7| 21703|     5|
|  1436|           7| 73763|     5|
|  1436|           7| 87408|     4|
|  1436|           7|102517|     5|
|  1436|           7|110084|     5|
|  1436|           7|135772|     5|
|  1436|           7|136449|     5|
|  1512|           5| 20711|     5|
|  1512|           5| 33496|     5|
|  1512|           5| 53149|     3|
|  1512|           5|107034|     3|
|  1512|           5|112977|     4|
|  2069|           7| 44119|     5|
|  2069|           7| 59067|     5|
|  2069|           7| 86486|     5|
|  2069|           7| 87658|     5|
|  2069|           7| 92043|     5|
|  2069|           7|104751|     5|
|  2069|           7|120983|     5|
|  2088|           7|  6853|     3|
+------+------------+------+------+
only showing top 20 rows



In [18]:
final_data.agg({"review_count": "min"}).collect()[0][0]

5

In [19]:
final_data.agg({"rating": "min"}).collect()[0][0]

'3'

In [20]:
final_data.write.csv("preprocessed_songsDataset.csv")